# ViT 2: Electric Transformaroo
We learned a few lessons from the previous experiment. First, and primarily, a larger model doesn't always mean better. We used the large imageNet transformer model, but that didn't actually lead to good predictions. Also, I think I screwed up a bunch of stuff, and the large model is taking way too long to train for me to iterate at all. We are going to switch to a smaller model. We are going to follow the ViTMAE method, use masked auto encodings for pretraining in a self-supervised manner on the entire training set, then fine tune on the prediction task. Guide found here: https://github.com/huggingface/transformers/blob/main/examples/pytorch/image-pretraining/run_mae.py 

This notebook has the fine-tuning process, fine tuning on Binary CE Loss.

In [1]:
# imports
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

import torch
print(torch.cuda.is_available())

from transformers import ViTFeatureExtractor, ViTForImageClassification, ViTMAEForPreTraining, ViTMAEConfig
from transformers import TrainingArguments, Trainer
from torchvision.transforms import RandomHorizontalFlip, RandomResizedCrop
from torchvision.transforms.functional import InterpolationMode

from sklearn.model_selection import train_test_split

True


In [2]:
from dataloader import *
from utils import *
from trainer import *

In [3]:
# CONSTANTS
FEATURE_EXTRACTOR_NAME = 'facebook/vit-mae-base'
VIT_MODEL_NAME = 'vit-mae-chexpert/'
TRAIN_SPLIT = 0.8
BATCH_SIZE = 48
LEARNING_RATE = 1.5e-3
LR_SCHEDULER_TYPE = "cosine"
WEIGHT_DECAY = 0.05
WARMUP_RATIO = 0.05
LOGGING_STRATEGY = "steps"
LOGGING_STEPS = 10
FP16 = True
EPOCHS = 10
EVALUATION_STRATEGY = "epoch"
EVAL_STEPS = 200
OUTPUT_DIR = './vit-mae-chexpert-fine-tuned-3'
REMOVE_UNUSED_COLUMNS = False
GRAD_ACCUM_STEPS = 5
DATALOADER_NUM_WORKERS = 4

In [4]:
feature_extractor = ViTFeatureExtractor.from_pretrained(FEATURE_EXTRACTOR_NAME, image_mean=[0.485, 0.456, 0.406], image_std=[0.229, 0.224, 0.225])

In [5]:
# set up our transforms
transforms = [
    RandomResizedCrop(feature_extractor.size, scale=(0.2, 1.0), interpolation=InterpolationMode.BICUBIC),
    RandomHorizontalFlip(),
]

In [6]:
np.random.seed(42)
train_df = pd.read_csv("ChexPert/train.csv")
train_df, eval_df = train_test_split(train_df, train_size=TRAIN_SPLIT)

train_dataset = ChexpertViTDataset("ChexPert/data", train_df, feature_extractor, transforms=transforms, classes=COMPETITION_TASKS,
    uncertainty_method="smooth", smoothing_lower_bound=0.55, smoothing_upper_bound=0.85)
eval_dataset = ChexpertViTDataset("ChexPert/data", eval_df, feature_extractor, classes=COMPETITION_TASKS,
    uncertainty_method="smooth", smoothing_lower_bound=0.55, smoothing_upper_bound=0.85)

In [7]:
model = ViTForImageClassification.from_pretrained(
    VIT_MODEL_NAME,
    num_labels=len(train_dataset.labels)
).to("cuda")

You are using a model of type vit_mae to instantiate a model of type vit. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at vit-mae-chexpert/ were not used when initializing ViTForImageClassification: ['decoder.decoder_layers.0.attention.output.dense.bias', 'decoder.decoder_layers.4.attention.attention.key.weight', 'decoder.decoder_layers.1.layernorm_before.bias', 'decoder.decoder_layers.6.attention.attention.key.weight', 'decoder.decoder_layers.0.attention.attention.key.weight', 'decoder.decoder_layers.4.attention.attention.query.bias', 'decoder.decoder_layers.1.output.dense.weight', 'decoder.decoder_layers.3.attention.attention.key.weight', 'decoder.mask_token', 'decoder.decoder_layers.5.attention.attention.query.bias', 'decoder.decoder_layers.3.attention.attention.value.weight', 'decoder.decoder_layers.2.attention.attention.query.weight', 'decoder.decoder_layers.2.layernorm_before.weight', 'decoder.decoder_layers.0.a

In [8]:
# set up training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    evaluation_strategy=EVALUATION_STRATEGY,
    num_train_epochs=EPOCHS,
    fp16=FP16,
    eval_steps = EVAL_STEPS,
    learning_rate=LEARNING_RATE,
    remove_unused_columns=REMOVE_UNUSED_COLUMNS,
    report_to="tensorboard",
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    logging_strategy=LOGGING_STRATEGY,
    logging_steps=LOGGING_STEPS,
    dataloader_num_workers=DATALOADER_NUM_WORKERS
)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

Using amp half precision backend


In [10]:
trainer.train()

/opt/conda/envs/chexpert/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 152878
  Num Epochs = 10
  Instantaneous batch size per device = 48
  Total train batch size (w. parallel, distributed & accumulation) = 240
  Gradient Accumulation steps = 5
  Total optimization steps = 6370


Epoch,Training Loss,Validation Loss,Accuracy,F1,Auc Atelectasis,Auc Cardiomegaly,Auc Consolidation,Auc Edema,Auc Pleural effusion
1,0.499800,0.478560,0.299326,0.399922,0.641550,0.757867,0.638014,0.762787,0.788048
2,0.493300,0.469987,0.305827,0.414086,0.653918,0.777125,0.645110,0.783250,0.805063
3,0.481700,0.471475,0.302472,0.386479,0.648077,0.775207,0.646301,0.781499,0.816498
4,0.471300,0.463352,0.315893,0.457207,0.661013,0.772222,0.649977,0.793999,0.817140
5,0.465800,0.457965,0.316155,0.467795,0.669215,0.803036,0.665254,0.804479,0.834070
6,0.455000,0.447963,0.329550,0.466538,0.675981,0.816666,0.670396,0.809163,0.836733
7,0.457300,0.445788,0.331568,0.464284,0.678726,0.817287,0.672642,0.812247,0.843804
8,0.447200,0.440336,0.337833,0.485155,0.683124,0.826002,0.675832,0.819827,0.848587
9,0.447800,0.438355,0.341634,0.496657,0.684915,0.829968,0.679873,0.821557,0.851640
10,0.447900,0.437575,0.342027,0.504517,0.685312,0.831033,0.680442,0.822031,0.852283


Saving model checkpoint to ./vit-mae-chexpert-fine-tuned-3/checkpoint-500
Configuration saved in ./vit-mae-chexpert-fine-tuned-3/checkpoint-500/config.json
Model weights saved in ./vit-mae-chexpert-fine-tuned-3/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 38149
  Batch size = 48
Saving model checkpoint to ./vit-mae-chexpert-fine-tuned-3/checkpoint-1000
Configuration saved in ./vit-mae-chexpert-fine-tuned-3/checkpoint-1000/config.json
Model weights saved in ./vit-mae-chexpert-fine-tuned-3/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 38149
  Batch size = 48
Saving model checkpoint to ./vit-mae-chexpert-fine-tuned-3/checkpoint-1500
Configuration saved in ./vit-mae-chexpert-fine-tuned-3/checkpoint-1500/config.json
Model weights saved in ./vit-mae-chexpert-fine-tuned-3/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 38149
  Batch size = 48
Saving model checkpoint to ./vit-mae-chexpert-

TrainOutput(global_step=6370, training_loss=0.4684260279267699, metrics={'train_runtime': 22749.6486, 'train_samples_per_second': 67.2, 'train_steps_per_second': 0.28, 'total_flos': 1.184713892621148e+20, 'train_loss': 0.4684260279267699, 'epoch': 10.0})

In [17]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 38149
  Batch size = 48


{'eval_loss': 0.40164998173713684,
 'eval_accuracy': 0.3934310204723584,
 'eval_runtime': 196.3203,
 'eval_samples_per_second': 194.32,
 'eval_steps_per_second': 4.05,
 'epoch': 3.0}

In [18]:
trainer.save_state()

In [11]:
trainer.save_model()

Saving model checkpoint to ./vit-mae-chexpert-fine-tuned-3
Configuration saved in ./vit-mae-chexpert-fine-tuned-3/config.json
Model weights saved in ./vit-mae-chexpert-fine-tuned-3/pytorch_model.bin


In [12]:
validation_df = pd.read_csv("ChexPert/valid.csv")
valid_dataset = ChexpertViTDataset("ChexPert/data", validation_df, feature_extractor, classes=COMPETITION_TASKS,
    uncertainty_method="smooth", smoothing_lower_bound=0.55, smoothing_upper_bound=0.85)

In [13]:
trainer.evaluate(eval_dataset=valid_dataset)

***** Running Evaluation *****
  Num examples = 202
  Batch size = 48


{'eval_loss': 0.4490750730037689,
 'eval_accuracy': 0.42574257425742573,
 'eval_f1': 0.4512820512820513,
 'eval_AUC_Atelectasis': 0.796482939632546,
 'eval_AUC_Cardiomegaly': 0.8231394830659537,
 'eval_AUC_Consolidation': 0.9020220588235295,
 'eval_AUC_Edema': 0.9047619047619047,
 'eval_AUC_Pleural Effusion': 0.9254981884057971,
 'eval_runtime': 2.123,
 'eval_samples_per_second': 95.15,
 'eval_steps_per_second': 2.355,
 'epoch': 10.0}